# 🧠 Study Buddy — RAG Student Notebook
Fill in the missing pieces to complete your own RAG pipeline using Gemini.


In [ ]:
# ✅ Step 1: Install dependencies
!pip install -q google-generativeai PyPDF2 faiss-cpu


In [ ]:
# ✅ Step 2: Import libraries
import google.generativeai as genai
from getpass import getpass
import PyPDF2
import faiss
import numpy as np
import re


In [ ]:
# ✅ Step 3: Configure Gemini API
# 👉 Fill in the variable name if needed
GEMINI_API_KEY = getpass("🔑 Enter your Gemini API key: ")
genai.configure(api_key=GEMINI_API_KEY)


In [ ]:
# ✅ Step 4: Upload a PDF or text file
# (This code is complete for you.)
from google.colab import files
uploaded = files.upload()

file_name = list(uploaded.keys())[0]
text = ""

if file_name.endswith(".pdf"):
    reader = PyPDF2.PdfReader(file_name)
    for page in reader.pages:
        text += page.extract_text() or ""
else:
    text = uploaded[file_name].decode("utf-8")

print(f"✅ Loaded {len(text)} characters from {file_name}")


In [ ]:
# ✅ Step 5: Chunking
# We provide the chunking logic. You choose the chunk_size if you want.

def split_text(text, chunk_size=1000, overlap=200):
    text = re.sub(r'\s+', ' ', text)
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

chunks = split_text(text)
print(f"📚 Split into {len(chunks)} chunks")


In [ ]:
# ✅ Step 6: Create embeddings + vector index
# ❗ Student TODO:
# 1. Set the embed model name
# 2. Call genai.embed_content
# 3. Build the FAISS index

embed_model = "<<< FILL IN MODEL NAME >>>"  # e.g., "models/gemini-embedding-001"

embeddings = []

for chunk in chunks:
    # 👉 TODO: generate an embedding for each chunk
    result = genai.embed_content(model=embed_model, content=chunk)
    embeddings.append( result["embedding"] )

embeddings = np.array(embeddings, dtype="float32")

# 👉 TODO: Initialize a FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

print("✅ Vector index built!")


In [ ]:
# ✅ Step 7: RAG Retrieval + Answering
# ❗ Student TODO:
# 1. Write the retrieve(query) function
# 2. Write the ask_study_buddy(query) function

def retrieve(query, k=3):
    # 👉 TODO: embed the query
    q_embed = genai.embed_content(model=embed_model, content=query)["embedding"]
    _, idx = index.search(np.array([q_embed], dtype="float32"), k)
    return [chunks[i] for i in idx[0]]

def ask_study_buddy(query):
    docs = retrieve(query)
    context = "\n\n".join(docs)

    # 👉 TODO: create your prompt template
    prompt = f"""
You are Study Buddy, a helpful learning assistant.
Use the context to answer the question clearly.

Context:
{context}

Question: {query}
"""

    model = genai.GenerativeModel("gemini-2.5-flash")
    response = model.generate_content(prompt)
    return response.text


In [ ]:
# ✅ Step 8: Test!
question = "<<< TYPE YOUR QUESTION HERE >>>"
print("🤔 Q:", question)
print("💡 A:", ask_study_buddy(question))
